In [1]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [3]:


gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [5]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [6]:
def make_embedding():
    inp = Input(shape=(100,100,3), name='input_image')

    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

    # Third block
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)


    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [7]:
embedding = make_embedding()

In [8]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2  (None, 46, 46, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 20, 20, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128)       26

In [9]:
# Siamese L1 Distance class
class L1Dist(Layer):

    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()

    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [10]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [11]:
# Reload model
siamese_model = tf.keras.models.load_model(r"C:\Users\2910p\Desktop\DESKTOP\LSML\siamesemodelv2.h5",
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [12]:
for image in os.listdir(r"C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification"):
    validation_img = os.path.join(r"C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification", image)
    print(validation_img)

C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0b607907-4f83-11ef-8458-346f24c8a482 - Copy.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0b607907-4f83-11ef-8458-346f24c8a482.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0b84e95b-4f83-11ef-88e2-346f24c8a482 - Copy.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0b84e95b-4f83-11ef-88e2-346f24c8a482.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0b9f12de-4f83-11ef-a1a0-346f24c8a482 - Copy.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0b9f12de-4f83-11ef-a1a0-346f24c8a482.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0be00e3d-4f83-11ef-a4d7-346f24c8a482 - Copy.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0be00e3d-4f83-11ef-a4d7-346f24c8a482.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification\0be7baec-4f83-11ef-b486-346f24c8a482 - Copy.jpg
C:\Users\2910p\Desktop\DESKTOP\LSML\application

In [13]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(r"C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification"):
        input_img = preprocess(r"C:\Users\2910p\Desktop\DESKTOP\LSML\application\input\input_img.jpg")
        validation_img = preprocess(os.path.join(r"C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification", image))

        # Make Predictions
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)

    # Detection Threshold: Metric above which a prediciton is considered positive
    detection = np.sum(np.array(results) > detection_threshold)

    # Verification Threshold: Proportion of positive predictions / total positive samples
    verification = detection / len(os.listdir(r"C:\Users\2910p\Desktop\DESKTOP\LSML\application\verification"))
    verified = verification > verification_threshold

    return results, verified

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]

    cv2.imshow('Verification', frame)

    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder
#         hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#         h, s, v = cv2.split(hsv)

#         lim = 255 - 10
#         v[v > lim] = 255
#         v[v <= lim] -= 10

#         final_hsv = cv2.merge((h, s, v))
#         img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

        cv2.imwrite(r"C:\Users\2910p\Desktop\DESKTOP\LSML\application\input\input_img.jpg", frame)
        # Run verification
        results, verified = verify(siamese_model, 0.9, 0.9)
        print(verified)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 61ms/step
True
1/1 [==============================] - 0s 57ms/step
False
